# 	IPython Companion Notebook for "Evaluation of Material Characterization Systems that Utilize a Two-Wire Transmission Line"

**Andrew Temme (temmeand@gmail.com) and Edward Rothwell (rothwell@egr.msu.edu), Michigan State University**

2015 IEEE AP-S Symposium on Antennas and Propagation and North American Radio Science Meeting  
Vancouver, British Columbia, Canada  
21 July 2015

Paper No. 4432  
Paper TUP-UA.1P.40: EVALUATION OF MATERIAL CHARACTERIZATION SYSTEMS THAT UTILIZE A TWO-WIRE TRANSMISSION LINE  
Session: TUP-UA.1P: Material, Modeling and Antenna Measurements  

This IPython notebook was used to perform calculations and create figures for the presented poster. Please cite this notebook as
    
    A. Temme and E. Rothwell. "Evaluation of Material Characterization Systems that Utilize a Two-Wire Transmission Line" companion IPython notebook, github.com/temmeand, 2015 URSI North American Radio Science Meeting, Vancouver, Canada. 21 July 2015.

This notebook is available as part of the Temme-and-Rothwell-URSI-2015 repo on [github.com/temmeand](github.com/temmeand) and [gitlab.msu.edu/temmeand](gitlab.msu.edu/temmeand).

# Notebook setup

In [1]:
# imports
# make sure that division is done as expected
from __future__ import division

# plotting setup
%matplotlib inline
import matplotlib.pyplot as plt

# for 3d graphs
from mpl_toolkits.mplot3d import axes3d

# for legends of combined fig types
import matplotlib.lines as mlines

# numerical functions
import numpy as np

# need some constants
from scipy.constants import epsilon_0, mu_0
from scipy.constants import c as c_0
from numpy import pi

# RF tools!
import skrf as rf

# version information
# %install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%version_information numpy, scipy, matplotlib

Software versions
Python 2.7.9 64bit [GCC 4.2.1 (Apple Inc. build 5577)]
IPython 3.0.0
OS Darwin 14.4.0 x86_64 i386 64bit
numpy 1.9.2
scipy 0.15.1
matplotlib 1.4.3
Mon Jul 27 10:30:01 2015 CDT

In [ ]:
# colormap
# you need the colormaps file from http://bids.github.io/colormap/
import colormaps as cmaps
cm_viridis = cmaps.viridis

# Configuration

Install the style sheet for plotting if needed.

In [ ]:
# install style sheet
# import shutil, matplotlib
# shutil.copy('./skrf-white-background-style.mplstyle', matplotlib.get_configdir()+'/stylelib')

Configuration settings

In [ ]:
# Plotting setup
doPlot = True
# doPlot = False

plt.style.reload_library()
plt.style.use('skrf-white-background-style')
cmap = plt.get_cmap(name='RdBu_r')
relcmap = cm_viridis
plt.rcParams['savefig.dpi'] = 300

# Run testing portions of code
# doTest = True
doTest = False

# save
# doSave = True
doSave = False

# Figure 1, 2, 3

Figures 1 and 2 are produced from HFSS models. These are available in the data directory as `data/sampleSideLength/model-for-images.hfss` and `data/pipe/pipe-3-color.hfss`, respectively. Figure 3 was created for the 2014 URSI presentation (Temme and Rothwell, 2014).

# Figure 4 and 5 - Electric potential, electric field, and magnetic field

## Electric Potential of a Two-Wire Transmission Line

The electric potential of a two-wire transmission line can be used to find the electric field of the transmission line. Image theory is used to find line charges that are equivalent to the charge on the conductors. This assumes that  the current is uniformly distributed around the conductor.

### Maths

We begin with Example 2.9 of Section 2.10 (p. 69) in Plonsey and Collin. This example deals mainly with a line charge outside of a conductor. Image theory is used to find an equivalent line charge inside of the conductor. Equation (2.76) can then be used to find the potential at any point outside of the conductor,

$$
\Phi(P) = \frac{q_1}{4\pi\epsilon}\ln\left(\frac{r_1}{r_2}\right)^2+C.\quad\quad (2.76)
$$

In this formulation, the computing the square of the log causes the potential to loose polarity. I prefer the form
$$
\Phi(P) = \frac{q_1}{2\pi\epsilon}\ln\left(\frac{r_1}{r_2}\right)+C.\quad\quad (2.76)
$$


The geometry and variables are given in Figure 2.24; $r_1$ is the distance from the charge outside of the conductor to the observation point $P$ and $r_2$ is the distance from the charge inside of the conductor to $P$. $r$ is the distance from the center of the conductor to $P$. The distances from the center of the conductor to the charge outside and inside of the conductor are $R_1$ and $R_2$, respectively.

By working through the example, we find that
$$
R_1=\frac{D}{2}+\sqrt{\frac{D^2}{4}-a^2}\quad\quad(2.86a)\\
R_2=\frac{D}{2}-\sqrt{\frac{D^2}{4}-a^2}\quad\quad(2.86b)
$$
where $D=R_1+R_2$ is the center-to-center distance between conductors of the two-wire transmission line and $a$ is the radius of the conductors.

From this geometry, we can find the coordinates of line charges. Line charge $q_1$ is located at $(-D/2+R_2,0)$ and $q_2$ is located at $(D/2-R_2)$. The distances $r_1$ and $r_2$ are then
$$
r_1 = \sqrt{(P_x+D/2-R_2)^2+P_y^2}\\
r_2 = \sqrt{(P_x-D/2+R_2)^2+P_y^2}.
$$

The value of $q$ may be determined if the voltage difference between the two conductors is known by solving
$$
V_2-V_1 = \frac{q}{2\pi\epsilon}\ln\left(\frac{D+\sqrt{D^2-4a^2}}{D-\sqrt{D^2-4a^2}}\right)\\
V_2-V_1 = \frac{q}{2\pi\epsilon}\ln\left(\frac{R_1}{R_2}\right).
$$

We should now be able to plot the potential given by Equation (2.76). The constant $C$ is determined by the ground reference.

### Initial Computation

Let us assume that there is a 1V potential between the conductors. The charge value is then
$$
1 = \frac{q}{2\pi\epsilon}\ln\left(\frac{R_1}{R_2}\right)\\
\Rightarrow q= \frac{2\pi\epsilon}{\ln(R_1/R_2)}
$$

We'll start by setting the constant integration constant to zero, $C=0$. This corresponds to a the midpoint between the conductors being the reference point.

## Electric Field

For a TEM wave, the electric field, $\vec{E}$, is given by
$$
\vec{E} = -\nabla_t \Phi\\
\Rightarrow E_x = \frac{-\partial \Phi}{\partial x},\quad\quad E_y = \frac{-\partial \Phi}{\partial y}
$$
where $\nabla_t$ is the gradient in the transverse ($\hat{x}$ and $\hat{y}$ for this problem) directions.

Define
$$
d = \frac{D}{2}-R_2\\
$$
and
$$
R_2 = \frac{D}{2}-\sqrt{\frac{D^2}{4}-a^2}
$$
as before.
The radial distances are then
$$
r_1 = \sqrt{(x+d)^2+y^2}\\
r_2 = \sqrt{(x-d)^2+y^2}.
$$
We may then express the potential as
$$
\Phi(x,y) = \frac{q}{2\pi\epsilon}\ln\left(\frac{r_1}{r_2}\right)\\
\Phi(x,y) = \frac{q}{2\pi\epsilon}\ln\left(\frac{\sqrt{(x+d)^2+y^2}}{\sqrt{(x-d)^2+y^2}}\right).
$$
Computation of the derivative gives
$$
\frac{-\partial\Phi}{\partial x} = \frac{-q}{2\pi\epsilon}\left[\frac{x+d}{(x+d)^2+y^2}-\frac{x-d}{(x-d)^2+y^2}\right]=E_x
$$
and
$$
\frac{-\partial\Phi}{\partial y} = \frac{-qy}{2\pi\epsilon}
\left(\frac{1}{(x+d)^2+y^2}-\frac{1}{(x-d)^2+y^2}\right)=E_y.
$$

In summary after substituting in $r_1$ and $r_2$, the electric field is
$$
E_x = \frac{-q}{2\pi\epsilon}\left(\frac{x+d}{r_1^2}-\frac{x-d}{r_2^2}\right)\\
E_y = \frac{-qy}{2\pi\epsilon}\left(\frac{1}{r_1^2}-\frac{1}{r_2^2}\right).
$$

## Magnetic Field

The magnetic field is calculated from the electric field,
$$
\vec{H} = \frac{\hat{k}\times\vec{E}}{\eta}\\
=\frac{\hat{z}\times(\hat{x}E_x+\hat{y}E_y)}{\eta}\\
=\hat{x}\frac{-E_y}{\eta}-\hat{y}\frac{-E_x}{\eta}=\hat{x}\frac{-E_y}{\eta}+\hat{y}\frac{E_x}{\eta}.
$$

The magnetic field is therefore
$$
H_x = \frac{-E_y}{\eta} = \frac{qy}{2\pi\epsilon\eta}\left(\frac{1}{r_1^2}-\frac{1}{r_2^2}\right)\\
H_y = \frac{E_x}{\eta} = \frac{-q}{2\pi\epsilon\eta}\left(\frac{x+d}{r_1^2}-\frac{x-d}{r_2^2}\right).
$$

## Problem Geometry

Specify the problem geometry and parameters.

In [ ]:
# Specs
# Geometry
D = 4
a = 1

# Electric specifications
Vdiff = 1

# Material
eps_r = 6

# Calculation Domain
# x, y extents
Pxmin = -8
Pxmax = 8
PxnumPts = 601
Pymin = -8
Pymax = 8
PynumPts = 601

# Plotting setup
doPlot = True
plotPad = 1.1
lowestdB = -100
cmap.set_bad(color='k')
relcmap.set_bad(color='k')

## Preflight Operations

In [ ]:
# Preflight Operations/Checks
# Error checking
assert a < D/2., "Wires overlap. Need a<D/2"
assert eps_r >= 1, "eps_r (%r), relative permittivity, must be 1 or larger." % eps_r
assert Pxmin < Pxmax, "Pxmin (%r) must be less than Pxmax (%r)" % (Pxmin, Pxmax)
assert Pymin < Pymax, "Pymin (%r) must be less than Pymax (%r)" % (Pymin, Pymax)
assert PxnumPts >=1, "PxnumPts (%r) must be greater than or equal to 1" % PxnumPts
assert PynumPts >=1, "PynumPts (%r) must be greater than or equal to 1" % PynumPts
assert lowestdB < 0, "lowestdB (%r) must be less than zero" % lowestdB


#----------
# Maths

# calculate constants
eps = eps_r * epsilon_0
eta = np.sqrt(mu_0/eps)

R1 = D/2 + np.sqrt(D**2/4-a**2)
R2 = D/2 - np.sqrt(D**2/4-a**2)
d = D/2-R2
assert R1 > 0, "R1 (%r) is not positive" % R1
assert R2 > 0, "R2 (%r) is not positive" % R2
assert R1 > R2, "R1 (%r) must be longer than R2 (%r)" % (R1, R2)
assert d > 0, "d (%r) is not positive" % d

q = 2*pi*eps*Vdiff/np.log(R1/R2)
A = q/(2*pi*eps)
# Vcond = abs(A*np.log(abs( (D/2-a+d)/(D/2-a-d) )))
Vcond = Vdiff/2

zeroLimit = Vcond*10**(lowestdB/20)

print "eps = ", eps
print "eta = ", eta
print "R1 = ", R1
print "R2 = ", R2
print "d = ", d
print "q = ", q
print "A = ", A
print "Vcond = ", Vcond
print "zeroLimit = ", zeroLimit

# generate x and y vectors
x = np.linspace(Pxmin,Pxmax,PxnumPts)
y = np.linspace(Pymin,Pymax,PynumPts)

# make arrays for the x, y at every point
x, y = np.meshgrid(x,y)

# create mask
left = (x+D/2)**2+y**2 < a**2
right = (x-D/2)**2+y**2 < a**2
inside = left | right
# x = np.ma.array(x,mask=inside)
# y = np.ma.array(y,mask=inside)

# calculate r1 and r2 distance for every pt
r1sq = (x+d)**2 + y**2
r2sq = (x-d)**2 + y**2
r1 = np.sqrt(r1sq)
r2 = np.sqrt(r2sq)
assert (r1sq >= 0).all(), "All elements of r1^2 are not positive"
assert (r2sq >= 0).all(), "All elements of r2^2 are not positive"
assert (r1 >= 0).all(), "All elements of r1 are not positive"
assert (r2 >= 0).all(), "All elements of r2 are not positive"

## Calculate Potential

In [ ]:
# find potential
Phi = A*np.log(r1/r2)

# set the potential inside of the conductors equal
# to the outside of the conductor
Phi[left] = -Vcond
Phi[right] = Vcond

# mask potential
Phi = np.ma.array(Phi,mask=inside)
# Phi[np.ma.where(np.ma.getmask(inside)==True)] = np.nan

print "Reference at x=0, the mid-point between the conductors"
print "Voltage on the conductors +/-", Vcond, "V"

## Calculate Electric and Magnetic Fields

In [ ]:
# Fields
# E field
Ex = -A*( (x+d)/r1sq - (x-d)/r2sq )
Ey = -A*y*(1/r1sq - 1/r2sq)
Emag = np.sqrt(Ex**2+Ey**2)
maxE = abs(-A*(1/((D/2-a)+d) - 1/((D/2-a)-d)))

# mask
Ex = np.ma.array(Ex,mask=inside)
Ey = np.ma.array(Ey,mask=inside)
Emag = np.ma.array(Emag,mask=inside)

#H field
Hx = -Ey/eta
Hy = Ex/eta
# np.sqrt does not accept masked arrays but still returns something
# this is messed up.
# see https://github.com/numpy/numpy/issues/5406
Hmag = np.ma.sqrt(Hx**2+Hy**2)
maxH = maxE/eta

# H is masked because it is derived from E fields

print "|Ex|_max = ", abs(Ex).max()
print "|Ey|_max = ", abs(Ey).max()
maxIndex = np.unravel_index(Emag.argmax(), Emag.shape)
print "|E|_max = %r at (%r, %r)"% ( Emag.max(), x[maxIndex], y[maxIndex])
print "Theoretical max ", maxE

## Fig 4. Visualize Fields

In [ ]:
if doPlot:
    fig, ax = plt.subplots()

    # line widths for stream plots
    ew = 10*Emag/maxE
    hw = 5*Hmag/maxH

    div = D-2*a
    print div
    # background heatmap for the potential
    im = ax.pcolormesh(x/div, y/div, Phi, cmap=cmap)
    cb = fig.colorbar(im)
    cb.set_label(r'Potential (V)')

    # Contour of magnetic field
    con = ax.contour(x/div, y/div, Hmag, 10,colors='k', linewidths=2, linestyles='dashed')

    # stream plots of fields
    stm = ax.streamplot(x/div, y/div, Ex, Ey, density=1, linewidth=ew, color='#b74331') 

    _= ax.set_xlim([Pxmin/div,Pxmax/div])
    _= ax.set_ylim([Pymin/div,Pymax/div])

    _= ax.set_xlabel('x/(D-2a)')
    _= ax.set_ylabel('y/(D-2a)')
    _= ax.set_aspect('equal')

    # legend
    eline = mlines.Line2D([],[], c='#b74331', marker='>', ms=5, label='E field')
    hline = mlines.Line2D([],[], c='k', ls='dashed', label='H field')
    _= ax.legend(handles=[eline, hline], loc=4)
    
    if doSave:
        fig.savefig("images/fields-and-potential.jpg", dpi=300, transparent=True)

## Fig. 5 Relative Electric Field
This one will probably throw a warning because of the labels.

In [ ]:
# calculate the lowest field strength for the given dB
print "Low field strength cutoff:", lowestdB, "dB"

# set values less than dB limit to the limit
EmagRelative = 20*np.log10(Emag/maxE)
nearLimit = EmagRelative < lowestdB
EmagRelative[nearLimit] = lowestdB

if doPlot:
    # plot potential
    fig, ax = plt.subplots()

    im = ax.pcolormesh(x/div, y/div, EmagRelative, cmap=relcmap)
    con = ax.contour(x/div,y/div,EmagRelative,10,cmap=relcmap, linewidths=2)

    _= ax.set_xlim([Pxmin/div,Pxmax/div])
    _= ax.set_ylim([Pymin/div,Pymax/div])

    cb = fig.colorbar(im)
    cb.set_label(r'E-Field Magnitude Relative to Maximum (dB)')
    
    _= ax.clabel(con, inline=False, fontsize=8, colors='k')

    _= ax.set_xlabel('x/(D-2a)')
    _= ax.set_ylabel('y/(D-2a)')
    _= ax.set_aspect('equal')
    
    if doSave:
        fig.savefig("images/EfieldRelative.jpg",dpi=300,transparent=True)

# Fig. 6

## Two-Wire Transmission Line Parameters
The transmission line parameters for a two-wire transmission line are

$$ R = \sqrt{\frac{\omega \mu_c}{2\pi^2 a^2 \sigma_c\left[1-(2a/D)^2\right]}}$$

$$L = \frac{\mu}{\pi}\cosh^{-1}\left(\frac{D}{2a}\right)$$

$$G = \frac{\omega\pi\epsilon^{\prime\prime}}{\cosh^{-1}(D/2a)}$$

$$C = \frac{\pi\epsilon^{\prime}}{\cosh^{-1}(D/2a)}$$

where $D$ is the center-to-center distance, $a$ is the radius of the wire, $\sigma_c$ is the conductivity of the wire, $\mu_c$ is the permeability of the wire (usually $\mu_c=\mu_0$), and finally $\mu$ and $\epsilon^c=\epsilon^\prime-j\epsilon^{\prime\prime}$ are the permeability and permittivity, respectively, of the material surrounding the wires.

The characteristic impedance of the transmission line is given by

$$Z_0 = \sqrt{\frac{R+j\omega L}{G+j\omega C}}$$

and the complex propagation constant is

$$\gamma = \alpha + j\beta = \sqrt{(R+j\omega L)(G+j\omega C)}.$$

In [ ]:
def twoWireParameters(freq, a, D, sigma_c=np.inf, mu_c=1,
                      eps_r = 1, tanDelta = 0, mu_r=1,
                      unitsScale=1):
    """
    Return R, L, G, C, Z_0 for a two-wire line
    
    Calculates and returns the resistance, impedance, conductance,
    and capacitance for a two-wire transmission line. Geometric and 
    electrical properties are given for the wire and the media in 
    which the wires are located.
    
    This function is/will be added to Scikit-rf.
    
    Parameters
    ----------
    freq : scalar
        Frequency at which the parameters should be calculated
    a : scalar
        Wire radius
    D : scalar
        center-to-center distance of the wires
    sigma_c : scalar
        Conductivity of the wires
    mu_c : scalar, optional
        Relative permeability of the wires. Default is 1.
    eps_r : scalar, optional
        Relative permittivity of the environment. Used as
        epsilon = eps_r*eps_0*(1-j*tanDelta). Default is 1.
    tanDelta : scalar, optional
        Loss tangent of the environment. Used as
        epsilon = eps_r*eps_0*(1-j*tanDelta). Default is 0.
    mu_r : scalar, optional
        Relative permeability of the environment. Default is 1
    unitsScale : scalar, optional if using meters
        Scaling factor for units. For mm, use 1e-3, for in use
        0.0254.
        
    Returns
    -------
        R : scalar
            Resistance of the two-wire transmission line
        L : scalar
            Impedance of the two-wire transmission line
        G : scalar
            Conductance of the two-wire transmission line
        C : scalar
            Capacitance of the two-wire transmission line
        Z_0 : scalar
            Complex impedance of the two-wire transmission line
    """
    #from __future__ import division
    from scipy.constants import epsilon_0, pi, mu_0
    
    # Scale dimensions
    a = a*unitsScale
    D = D*unitsScale
    eps_sgl = epsilon_0*eps_r
    eps_dbl = epsilon_0*eps_r*tanDelta
    
    omega=2*pi*freq
    #delta_cond = 1/np.sqrt(const.pi*freq*sigma_c)
    
    if  ( (type(D) is float and D <= (2*a))
         or (type(D) is np.ndarray and np.any(D <= (2*a))) ):
        raise ValueError('Center to center spacing is less than twice the wire radius')
    
    invCosh = np.arccosh(D/(2*a))
    
    R = np.sqrt(omega*mu_c*mu_0/(2*pi**2*a**2*sigma_c*(1-(2*a/D)**2)))
    G = (pi*omega*eps_dbl)/invCosh
    L = mu_0/(pi)*invCosh
    C = pi*eps_sgl/invCosh
    
    if type(freq) is np.ndarray:
        G = G*ones(freq.shape)
        L = L*ones(freq.shape)
        C = C*ones(freq.shape)
    
    Z_0 = np.sqrt((R+1j*omega*L)/(G+1j*omega*C))

    return R, L, G, C, Z_0

## Fig 6. (a), (b) Gamma vs eps and tanDelta

In [ ]:
# Variable eps_r, copper transmission line

# Computation domain
eps_r = np.linspace(1,10,201)
tanDelta = np.linspace(0,0.2,201)

# create a grid
eps_r, tanDelta = np.meshgrid(eps_r, tanDelta)

Dovera = 5

# define the radius and seperation distance
a = 1e-3/2 # radius
D = Dovera*a # seperation distance

# define other parameters
freq = 500e6
# tanDelta = 0
unitsScale = 1
sigma_c = 5.96*10**7
# sigma_c = np.inf

omega = 2*pi*freq

R, L, G, C, Z_c = twoWireParameters(freq, a, D, eps_r=1, sigma_c=sigma_c, unitsScale=unitsScale)
gamma_c = np.sqrt((R+1j*omega*L)*(G+1j*omega*C))

R, L, G, C, Z = twoWireParameters(freq, a, D, eps_r=eps_r, tanDelta=tanDelta, 
                                  sigma_c=sigma_c, unitsScale=unitsScale)
gamma = np.sqrt((R+1j*omega*L)*(G+1j*omega*C))

alpha = gamma.real/gamma_c.real
beta = gamma.imag/gamma_c.imag

if doPlot:
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    surf = ax.plot_surface(eps_r, tanDelta, alpha, rstride=10, cstride=10, 
                           alpha=0.5, cmap=relcmap, lw=0, antialiased=False)

    cset = ax.contour(eps_r, tanDelta, alpha, 5, zdir='x', offset=eps_r.min(), cmap=relcmap)
    cset = ax.contour(eps_r, tanDelta, alpha, 5, zdir='y', offset=tanDelta.max(), cmap=relcmap)
    cset = ax.contourf(eps_r, tanDelta, alpha, 5, zdir='z', offset=alpha.min(), cmap=relcmap)

    cbar = fig.colorbar(surf)
    cbar.solids.set_edgecolor("face")

    _ = ax.set_xlabel(r'$\epsilon_r$')
    _ = ax.set_ylabel(r'$\tan\delta$')
    _ = ax.set_zlabel(r'$\alpha/\alpha_0$ with copper')
    if doSave:
        fig.savefig("images/alpha.eps",transparent=True)
        fig.savefig("images/alpha.svg",transparent=True)
        fig.savefig("images/alpha.jpg",dpi=300,transparent=True)

    # ---
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    surf = ax.plot_surface(eps_r, tanDelta, beta, rstride=10, cstride=10, 
                           alpha=0.8, cmap=relcmap, lw=0, antialiased=False)
    cset = ax.contour(eps_r, tanDelta, beta, 5, zdir='x', offset=eps_r.min(), cmap=relcmap)
    cset = ax.contour(eps_r, tanDelta, beta, 5, zdir='y', offset=tanDelta.max(), cmap=relcmap)
    cset = ax.contourf(eps_r, tanDelta, beta, 5, zdir='z', offset=beta.min(), cmap=relcmap)
    
    cbar = fig.colorbar(surf)
    cbar.solids.set_edgecolor("face")

    _ = ax.set_xlabel(r'$\epsilon_r$')
    _ = ax.set_ylabel(r'$\tan\delta$')
    _ = ax.set_zlabel(r'$\beta/\beta_0$ with copper')
    if doSave:
        fig.savefig("images/beta.eps",transparent=True)
        fig.savefig("images/beta.svg",transparent=True)
        fig.savefig("images/beta.jpg",dpi=300,transparent=True)

## Fig 6. (c) Thickness for Desired Phase Change

In [ ]:
# thickness for desired phase change vs eps_r
f = 1.5e9
a = 1e-3
D = 10*a
copper = 5.96e7
pec = np.inf
eps_r = np.linspace(2,10,201)
tand = 0.2
# eps_r = 2.4
# tand = np.linspace(0,0.2,201)

phaseDiff = 1
phaseDiff = np.deg2rad(phaseDiff)

omega = 2*pi*f

R, L, G, C, Z0 = twoWireParameters(f, a, D, copper, 1, 1, 0)
gamma_0 = np.sqrt((R+1j*omega*L)*(G+1j*omega*C))
beta_0 = gamma_0.imag

R, L, G, C, Z0 = twoWireParameters(f, a, D, copper, 1, eps_r, tand)
gamma_1 = np.sqrt((R+1j*omega*L)*(G+1j*omega*C))
beta_1 = gamma_1.imag

length = phaseDiff/(beta_1-beta_0)
lw = length*f/(1e-3*1e9*phaseDiff*180/pi)

if doPlot:
    fig, ax = plt.subplots()
    
    lin = ax.plot(eps_r,lw)
    
    _= ax.set_xlabel(r'$\epsilon_r$')
    _= ax.set_ylabel(r'$l\omega/(1mm\ 1GHz\ \Delta\phi^\circ)$')

# --------------------------------------------
# now calculate the 2nd axis

# thickness for desired phase change vs tandelta
f = 1.5e9
a = 1e-3
D = 10*a
copper = 5.96e7
pec = np.inf
# eps_r = np.linspace(2,10,201)
# tand = 0.2
eps_r = 2.4
tand = np.linspace(0,.2,201)

phaseDiff = 1
phaseDiff = np.deg2rad(phaseDiff)

omega = 2*pi*f

R, L, G, C, Z0 = twoWireParameters(f, a, D, copper, 1, 1, 0)
gamma_0 = np.sqrt((R+1j*omega*L)*(G+1j*omega*C))
beta_0 = gamma_0.imag

R, L, G, C, Z0 = twoWireParameters(f, a, D, copper, 1, eps_r, tand)
gamma_1 = np.sqrt((R+1j*omega*L)*(G+1j*omega*C))
beta_1 = gamma_1.imag

length = phaseDiff/(beta_1-beta_0)
lw = length*f/(1e-3*1e9*phaseDiff*180/pi)

if doPlot:
    axT = ax.twinx()
    axR = axT.twiny()
    
    lin = axR.plot(tand,lw, ls='-.',c=plt.rcParams['axes.color_cycle'][1])
    
    _= axR.set_xlabel(r'$\tan\delta$')
    _= axT.set_ylabel(r'$l\omega/(1mm\ 1GHz\ \Delta\phi^\circ)$')
    
    # legend
    bottomc = plt.rcParams['axes.color_cycle'][0]
    topc = plt.rcParams['axes.color_cycle'][1]
#     bottom = mlines.Line2D([],[], c=bottomc, label=r'$\epsilon_r$')
#     top = mlines.Line2D([],[], c=topc, ls='-.', label=r'$\tan\delta$')
#     _= ax.legend(handles=[bottom, top], loc='upper right')
    
    # adjust so that the ticks/grids line up
    _= ax.set_ylim([0.2,1.6])
    _= axR.set_ylim([1.490,1.525])
    
    # arrows pointing to axes

    _= ax.annotate("", xy=(4,.4), xytext=(5, 0.6),
                   arrowprops=dict(arrowstyle="->", color=bottomc,lw=1))
    _= ax.text(3.8,.6,r'$\tan\delta=0.2$',color=bottomc)
    _= ax.annotate("", xy=(9,1), xytext=(8, .8),
                   arrowprops=dict(arrowstyle="->", color=topc,lw=1))
    _= ax.text(8.5,.8, r"$\epsilon_r=2.4$", color=topc)
    
    if doSave:
        fig.savefig("images/1deglength.eps",transparent=True)
        fig.savefig("images/1deglength.svg",transparent=True)
        fig.savefig("images/1deglength.jpg",dpi=300,transparent=True)

## Fig 6. (d) Input Impedance

In [ ]:
if doPlot:
    freq = 1e9
    a = .125/2  # wire radius
    D = np.linspace(2*a+1e-6, 10*a, 200)  # center to center distance
    sigma_c = np.inf  # conductivity of wire
    mu_c = 1  # permeability of wire
    eps_r = 1  # permittivity of surrounding media
    tanDelta = 0  # loss tangent of surrounding media
    mu_r = 1  # permeability of surrounding media
    unitsScale = 0.0254  # multiply by to get meters (inches:0.0254)

    R, L, G, C, Z = twoWireParameters(freq, a, D, sigma_c, mu_c, eps_r, tanDelta, mu_r, unitsScale)

    fig, ax = plt.subplots()

    ax.plot(D/a,abs(Z))
    ax.set_xlabel(r'$D/a$')
    ax.set_ylabel(r'Characteristic Impedance/$\Omega$')
    ax.text(5,100,"PEC wires in free space\n\nTemme et al. 2014")

    if doSave:
        fig.savefig("images/zIn.eps",transparent=True)
        fig.savefig("images/zIn.svg",transparent=True)
        fig.savefig("images/zIn.jpg",dpi=300,transparent=True)

# Fig. 7 Radiation Resistance

In [ ]:
def radiationResistance(freq, a, D, sigma_c=np.inf, mu_c=1,
                      eps_r = 1, tanDelta = 0, mu_r=1, length=1,
                      zLoad = 50, unitsScale=1):
    """
    Return the radiation resistance of a two-wire line of given length
    
    Returns the radiation resistance of a two-wire transmission line.
    Geometric and electrical properties are given for the wire and the
    media in which the wires are located.
    
    This function does not use an ``effective'' seperation distance as
    would be needed when the wires are close
    
    Parameters
    ----------
    freq : scalar
        Frequency at which the parameters should be calculated
    a : scalar
        Wire radius
    D : scalar
        center-to-center distance of the wires
    sigma_c : scalar
        Conductivity of the wires
    mu_c : scalar, optional
        Relative permeability of the wires. Default is 1.
    eps_r : scalar, optional
        Relative permittivity of the environment. Used as
        epsilon = eps_r*eps_0*(1-j*tanDelta). Default is 1.
    tanDelta : scalar, optional
        Loss tangent of the environment. Used as
        epsilon = eps_r*eps_0*(1-j*tanDelta). Default is 0.
    mu_r : scalar, optional
        Relative permeability of the environment. Default is 1.
    length : scalar, optional
        The length of the transmission line. Default is 1.
    zLoad : complex scalar, optional
        Complexe impedance of the load. Default is 50 ohms.
    unitsScale : scalar, optional if using meters
        Scaling factor for units. For mm, use 1e-3, for in use
        0.0254.
        
    Returns
    -------
        rRad : scalar
            Radiation resistance of a two-wire tranmission line
    """
    #from __future__ import division
    import scipy.constants as const

    # Get the circuit parameters of the line
    R, L, G, C, z0 = twoWireParameters(freq, a, D, sigma_c, mu_c,
                      eps_r, tanDelta, mu_r, unitsScale)
    
    # Scale dimensions
    a = a*unitsScale
    D = D*unitsScale
    length = length*unitsScale
    eps_sgl = const.epsilon_0*eps_r
    eps_dbl = const.epsilon_0*eps_r*tanDelta
    
    omega=2*pi*freq
    
    eta1 = np.sqrt((const.mu_0*mu_r)/(const.epsilon_0*eps_r))
    beta1 = omega*np.sqrt(const.mu_0*mu_r*const.epsilon_0*eps_r)
    gamma = np.sqrt((R+1j*2*pi*freq * L)*(G+1j*2*pi*freq*C))
    alpha = gamma.real
    beta = gamma.imag
    thetaP = np.arctanh(zLoad/z0)
    rhoL = thetaP.real
    twoBetaLen = 2*beta1*length
    rRad = ( eta1/(4*pi)*(beta1*D)**2 * np.cosh(alpha*length+2*rhoL)
            / abs(np.cosh(gamma*length+thetaP))**2
            *(np.cosh(alpha*length) - np.sin(twoBetaLen)/twoBetaLen)
            )
    
    # rRad = eta1/(2*pi)*beta1**2*D**2*(1-sin(2*beta*length)/(2*beta*length))
    # rRad = 30*beta1**2*D**2*(1-sin(twoBetaLen)/twoBetaLen)
    # print "num:", np.cosh(alpha*length[0]+2*rhoL)
    # print "denom:", abs(np.cosh(gamma*length[0]+thetaP))**2
    # print "alpha * length:", alpha*length[0]
    # print "as+2p:", alpha*length[0]+rhoL*2
    return rRad

In [ ]:
# (non)resonant cases
eta = np.sqrt(mu_0/epsilon_0)
betaL = np.linspace(1e-9,8*pi/2,200)
res = eta/4/pi*(1-np.sin(2*betaL)/(2*betaL))
non = res*2
lim = 30*np.ones(np.shape(betaL))

lineColor = plt.rcParams['axes.color_cycle'][0]
color2 = plt.rcParams['axes.color_cycle'][1]

# ----------------------------------------------------
# Z0/2
freq = 1e9
a = 1e-3  # wire radius
D = 10*a  # center to center distance
sigma_c = np.inf  # conductivity of wire
mu_c = 1  # permeability of wire
eps_r = 1  # permittivity of surrounding media
tanDelta = 0  # loss tangent of surrounding media
mu_r = 1  # permeability of surrounding media
unitsScale = 1  # multiply by to get meters (inches:0.0254)
R, L, G, C, Z = twoWireParameters(freq, a, D, sigma_c, mu_c, eps_r, tanDelta, mu_r, unitsScale)
gamma = np.sqrt((R+1j*2*pi*freq * L)*(G+1j*2*pi*freq*C))
beta = gamma.imag

betaLen = np.linspace(1e-6,4*pi,100)
length = betaLen/beta

rRad = radiationResistance(freq, a, D, length=length, zLoad=Z*.5, unitsScale=1)

betaFree = 2*pi*freq*np.sqrt(epsilon_0*mu_0)

if doPlot:
    fig, ax1 = plt.subplots()

    ax1.plot(betaL, res, label='Resnonant')
    ax1.plot(betaL, res*2, ls=':', label='Nonresonant/2')
#     ax1.plot(betaL, lim, ls='--', color=lineColor,label='Limit')
    lin = ax1.plot(betaLen,rRad/(betaFree**2*(D)**2),ls='--',label=r'$Z_0/2$')

    ax1.legend(loc='lower right', ncol=3)
    
    ax1.set_xlabel(r"$\beta l$")
    ax1.set_ylabel(r"Radiation resistance$/(\eta_{1}\beta_1^2D^2\ \Omega)$")
    
    ax1.set_xlim([0,8*pi/2])
    ax1.set_xticks([0,pi/2,pi,3*pi/2,2*pi,5*pi/2,3*pi,7*pi/2,4*pi])
    ax1.set_xticklabels(["",r"$\pi/2$",r"$\pi$",r"$3\pi/2$",r"$2\pi$",
                         r"$5\pi/2$",r"$3\pi$",r"$7\pi/2$",r"$4\pi$"])
    # equations
#     ax1.text(pi,20,r'$R^{rad}=\frac{\eta_1}{4\pi}\beta_1^2 D^2 \frac{\cosh(\alpha s + 2\rho_L)}{\left|\cosh(\gamma s + \theta^\prime_L)\right|^2}\left(\cosh(\alpha s)-\frac{\sin(2\beta s)}{2\beta s}\right)$')
#     ax1.text(pi,16,r'$\theta^\prime_L = \rho_L+j\Phi_L^\prime=\tanh^{-1}\left(\frac{Z_L}{Z_0}\right)$')
#     ax1.text(pi,11,r'$R_{res}^{rad}=\frac{\eta_1}{4\pi}\beta_1^2 D^2 \sec^2(\beta s+\Phi^\prime_L)\left(1-\frac{\sin(2\beta_1 s)}{2\beta_1 s}\right)=\frac{\eta_1}{4\pi}\beta_1^2D^2$')
#     ax1.text(pi, 7,r'$R_{non}^{rad}=\frac{\eta_1}{4\pi}\beta_1^2 D^2 \left(\cosh(\alpha s)-\frac{\sin(2\beta s)}{2\beta s}\right)=\frac{\eta_1}{2\pi}\beta_1^2D^2$')
#     ax1.text(pi, 3,'King, 1955')

    if doSave:
        fig.savefig("images/rRad.eps",transparent=True)
        fig.savefig("images/rRad.svg",transparent=True)
        fig.savefig("images/rRad.jpg",dpi=300,transparent=True)

# Fig 8 Sample Side Length

In [ ]:
# sample side length
samples = rf.NetworkSet(rf.read_all('data/sampleSideLength',contains="scale"))
samples.sort()
for index, s in enumerate(samples):
    print "[%2r] %r" % (index, s)
    
ref = rf.Network('data/sampleSideLength/scale-20.s2p')
if doPlot:
    fig, ax = plt.subplots()

ix = np.array([0,4,6,7,8,9,10,11,12,1,2,3,5])
for a in ix:
    s = samples[a]
    diff = s.s21.s_deg_unwrap-ref.s21.s_deg_unwrap
    diff = diff[:,0,0]
    if doPlot:
        lin = ax.plot(ref.f/1e9,diff,label=s.name)

if doPlot:
    _= ax.set_xlabel('Frequency (GHz)')
    _= ax.set_ylabel(r'$\angle S_{i} - \angle S_{ref}$')

    _= ax.legend()
    _= ax.set_xlim([0.5,6])
    _= ax.set_ylim([-1, 30])
    
    if doSave:
        fig.savefig("images/sampleSideLength.eps",transparent=True)
        fig.savefig("images/sampleSideLength.svg",transparent=True)
        fig.savefig("images/sampleSideLength.jpg",dpi=300,transparent=True)

# Fig 9 Shielded Pair

In [ ]:
# shielded pair
b = 1
a1 = b/10
a2 = np.linspace(2*(b-2*a1), 10*(b-2*a1), 201)
factor=.99

k = 2*np.log(b*(a2**2-b**2/4)/(a1*(a2**2+b**2/4)))
f = 2*np.log(b/a1)*np.ones(a2.shape)
lim = f*factor

if doPlot:
    fig, ax = plt.subplots()
    lin = ax.plot(a2/(b-2*a1),k,label="Shielded")
    lin = ax.plot(a2/(b-2*a1),f,label="Unshielded")
    lin = ax.plot(a2/(b-2*a1),lim,ls='-.',label=str(factor*100)+'%')
    _= ax.legend()
    _= ax.set_xlabel(r'Distance away/gap')
    _= ax.set_ylabel(r'$k$ factor')
    if doSave:
        fig.savefig("images/shieldedPair.eps",transparent=True)
        fig.savefig("images/shieldedPair.svg",transparent=True)
        fig.savefig("images/shieldedPair.jpg",dpi=300,transparent=True)

# References

1. R. W. P. King, Transmission-Line Theory. New York: McGraw-Hill, 1955.
1. J. R. Whinnery and T. Van Duzer with Simon. Ramo, Fields and Waves in Communication Electronics, 3rd ed. New York: Wiley, 1994.
1. R. Plonsey and R. E. Collin, Principles and applications of electromagnetic fields. New York: McGraw-Hill, 1961.
1. Hewlett-Packard, “Measuring Dielectric Constant with the HP 8510 Network Analyzer: The Measurement of both Permittivity and Permeability of Solid Materials,” Hewlett-Packard, Product Note 8510-3, Aug. 1985.
1. A. Temme and E. Rothwell. "Material Characterization using a Two-Wire Transmission Line," 2014 URSI North American Radio Science Meeting, Memphis, Tennessee, USA.